In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
import polars as pl
import sys
sys.path.append('.') 
import upsert_utils

In [3]:
duckdb.sql("""
attach '../../db.sqlite3';
use db;
""")

In [4]:
# duckdb.sql("""
# rollback;
# """)

In [5]:
duckdb.sql("""
begin transaction;
""")

In [6]:
ob_source_countries = pl.DataFrame([
    dict(
        ProdCode_Value='QCELL-Q_TRON_BLK_M_G2_H__430',
        AssignedCostPercentage_Value=None,
        CountryOfManufacture_Value='US', CountryOfManufactureIsNotPFE_Value=True,
        CountryOfOwnershipforPFE_Value='US', CountryOfOwnershipIsNotPFE_Value=True,
    ),
    dict(
        ProdCode_Value='QCELL-Q_TRON_BLK_M_G2_H1__AC_430',
        AssignedCostPercentage_Value=None,
        CountryOfManufacture_Value='US', CountryOfManufactureIsNotPFE_Value=True,
        CountryOfOwnershipforPFE_Value='US', CountryOfOwnershipIsNotPFE_Value=True,
    ),
]).with_columns(csv_row_id=1 + pl.int_range(pl.len()))
ob_source_countries

ProdCode_Value,AssignedCostPercentage_Value,CountryOfManufacture_Value,CountryOfManufactureIsNotPFE_Value,CountryOfOwnershipforPFE_Value,CountryOfOwnershipIsNotPFE_Value,csv_row_id
str,null,str,bool,str,bool,i64
"""QCELL-Q_TRON_BLK_M_G2_H__430""",null,"""USA""",true,"""USA""",true,1
"""QCELL-Q_TRON_BLK_M_G2_H1__AC_4…",null,"""US""",true,"""US""",true,2


In [7]:
ob_product_ids = duckdb.sql("""
select
    ob_source_countries.csv_row_id,
    id as product_id,
from server_product
join ob_source_countries on server_product.ProdCode_Value = ob_source_countries.ProdCode_Value
""")
ob_product_ids.show(max_width=120)

┌────────────┬────────────┐
│ csv_row_id │ product_id │
│   int64    │   int64    │
├────────────┼────────────┤
│          1 │      12933 │
│          2 │      12934 │
└────────────┴────────────┘



In [8]:
ob_source_country_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        * exclude(ProdCode_Value),
    from ob_source_countries
    """),
    target_table_name='server_sourcecountry',
)
ob_source_country_ids

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│          1 │     1 │
│          2 │     2 │
└────────────┴───────┘

In [9]:
upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        ob_source_country_ids.csv_row_id,
        product_id,
        ob_source_country_ids.id as sourcecountry_id,
    from ob_source_country_ids
    join ob_source_countries using (csv_row_id)
    join ob_product_ids using (csv_row_id)
    """),
    target_table_name='server_product_SourceCountries',
)

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│          2 │     1 │
│          1 │     2 │
└────────────┴───────┘

In [10]:
duckdb.sql("""
commit;
""")